In [7]:
import time
import pandas as pd
from py2neo import Graph, Node, Relationship

In [5]:
def query_to_df(query, graph):
    print("Starting query...", end=" ")
    query_start_time = time.time()
    df = graph.run(query).to_data_frame()
    print("Done ({:.2f} minutes).".format((time.time()-query_start_time)/60))
    return df

In [6]:
graph = Graph( "bolt://matlaber5.media.mit.edu:7687", auth=('neo4j','myneo'))
print("Connected to graph database with {:,} nodes and {:,} relationships!".format(
    graph.database.primitive_counts['NumberOfNodeIdsInUse'], 
    graph.database.primitive_counts['NumberOfRelationshipIdsInUse']))

Connected to graph database with 278,432,359 nodes and 1,844,501,832 relationships!


In [4]:
top_5 = ['Cell', 'Nature', 'Nature Biotechnology','Proceedings of the National Academy of Sciences of the United States of America','Science']
top_10 = ['Cell', 'Nature', 'Nature Biotechnology','Proceedings of the National Academy of Sciences of the United States of America','Science', 'Journal of the American Chemical Society', 'JAMA', 'The New England Journal of Medicine', 'Nature Genetics', 'Neuron']
top_42 = ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']

In [13]:
import pandas as pd
df_impact_rank = pd.read_csv('~/Workspace/UROP/scaling-science/notebooks/Collaboration/Data/impact_rank_1000.csv', 
                             encoding = "UTF-8")
top_profs = []
for p in list(df_impact_rank.head(200)['avg_PR']):
    if p in list(df_impact_rank.head(200)['avg_AR']):
        top_profs.append(p)
print(top_profs)
print(len(top_profs))

['Ulrich K. Laemmli', 'Stephen F. Altschul', 'David J. Lipman', 'Tom Maniatis', 'Oliver H. Lowry', 'Joseph Sambrook', 'Lars Hedin', 'Kary B. Mullis', 'William H. Stein', 'Randall K. Saiki', 'Stanford Moore', 'Theophil Staehelin', 'Mark S. Guyer', 'Saul A. Teukolsky', 'William T. Friedewald', 'Paul Meier', 'Amos Tversky', 'Sidney P. Colowick', 'A. F. Huxley', 'Harry Towbin', 'Koichiro Tamura', 'W. James Kent', 'Lucinda Fulton', 'Arian Smit', 'Torsten N. Wiesel', 'Jeffrey Vieira', 'Folke Skoog', 'Marilyn Bergner', 'Eugene W. Myers', 'Thomas R. Dawber', 'Desmond G. Higgins', 'Alan Coulson', 'A. L. Hodgkin', 'Leon Festinger', 'Carole A. Cull', 'G. E. Palade', 'Masatoshi Nei', 'C Redmond', 'Rebecca DerSimonian', 'Marjorie B. Lees', 'Terrence S. Furey', 'Eli Robins', 'Richard Helm', 'I. M. Stratton', 'Josep F. Abril', 'Julie D. Thompson', 'Katherine M. Flegal', 'Daniel Kahneman', 'Joseph L. Fleiss', 'Russell Higuchi', 'B. G. Barrell', 'Michael C. Wendl', 'E. Kellenberger', 'John H. Holland',

In [17]:
f =  open('/home/brendan/Workspace/UROP/scaling-science/notebooks/Collaboration/Data/impact_1000.txt', 'w')
f.write('[')
for prof in top_profs[:-1]:
    f.write("'" + prof + "', ")
f.write("'" + top_profs[-1] + "']")
f.close()

In [ ]:
# Average num_authors
query = """call apoc.periodic.iterate(
    "MATCH (b:Top42Author)-[:AUTHORED]-(pub:Quanta)
    RETURN pub, COUNT(DISTINCT b) as num_authors",
    "SET pub.num_authors = num_authors", {batchSize:10000, iterateList:true, parallel:true})
    """
df = query_to_df(query, graph)
df

query = """
call apoc.export.csv.query(
"MATCH (a:Top42Author)-[:AUTHORED]->(q:Quanta)
	WHERE q.venue in ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']
    RETURN avg(q.num_authors)"
, "/data/csv/avg_num_authors.csv", {batchSize:10000, iterateList:true, parallel:true})
    """
df = query_to_df(query, graph)
df

Starting query... 

In [19]:
# Disruption
query = """
call apoc.export.csv.query(
"MATCH (a:Top42Author)
WHERE a.last_author_yr > 2013 

MATCH (a)-[:AUTHORED]->(q:Quanta)<-[:AUTHORED]-(b:Author)
WHERE q.venue in ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']
    AND q.year < a.last_author_yr
    AND b.name in ['Timothy Fennell', 'Parnesh Raniga', 'Jeffrey C. Lee', 'James I. Cleeman', 'Todd M. Lowe', 'Amos Tversky', 'Eli Robins', 'Stephen T. Sherry', 'Desmond G. Higgins', 'John G. Kirkwood', 'Jane Peterson', 'Michael B. Eisen', 'G. E. Palade', 'Wayne J. Higgins', 'Sean R. Eddy', 'Torsten N. Wiesel', 'Jacques Ferlay', 'Nan M. Laird', 'Aravind Subramanian', 'Nathan Mantel', 'LaDeana W. Hillier', 'Richard Helm', 'Arian Smit', 'Anthony R. Kerlavage', 'Oliver H. Lowry', 'Carl-Magnus Rudenstam', 'P. Armitage', 'Nj Bell', 'Richard H. Creech', 'Russell Higuchi', 'Tobias Doerks', 'Theophil Staehelin', 'Edwin M. Southern', 'Sidney P. Colowick', 'Nicoletta Sacchi', 'J. Gillis', 'Stephen F. Altschul', 'Leon Festinger', 'I. M. Stratton', 'Julie D. Thompson', 'Thomas R. Dawber', 'Stanford Moore', 'E. Kellenberger', 'Kary B. Mullis', 'David A. Hungerford', 'Koichiro Tamura', 'Marshal F. Folstein', 'Tom Maniatis', 'Steven E. Scherer', 'C Redmond', 'Daniel Kahneman', 'Margaret D. Carroll', 'Elizabeth Lacy', 'David S. Johnson', 'Tavia Gordon', 'Ian Korf', 'David J. Dooling', 'R. A. Fisher', 'Marjorie B. Lees', 'Harry Towbin', 'William H. Stein', 'Paul Meier', 'Mark S. Guyer', 'Terrence S. Furey', 'Kara Dolinski', 'John P. Huelsenbeck', 'Jane G. Schaller', 'Susan E. Folstein', 'Randall K. Saiki', 'Ulrich K. Laemmli', 'William T. Friedewald', 'Michael L. Watson', 'F. Sanger', 'David P. Bartel', 'Rebecca DerSimonian', 'Felsenfeld Al', 'Michael C. Wendl', 'Saul A. Teukolsky', 'Parkhurst A. Shore', 'Josep F. Abril', 'David R. Bentley', 'N. E. Day', 'Kevin McKernan', 'Qui-Lim Choo', 'Tim R. Mosmann', 'Joseph L. Fleiss', 'Carole A. Cull', 'Thomas D. Schmittgen', 'Mark T. Ross', 'W. James Kent', 'Catherine A. Ball', 'Lucinda Fulton', 'Jeffrey Vieira', 'Patrick O. Brown', 'Kenneth J. Livak', 'Granger G. Sutton', 'David J. Lipman', 'Deanna M. Church', 'John E. Ware', 'Michael C. Zody', 'A. F. Huxley', 'Renaud Capdeville', 'A. L. Hodgkin', 'Sidney J. Cutler', 'Eugene W. Myers', 'Alan Coulson', 'Joseph Sambrook', 'Lars Hedin', 'Martin Gibson', 'Martin Ringwald', 'Marilyn Bergner', 'Masatoshi Nei', 'William Haenszel', 'D. R. Cox', 'R W Davis', 'Folke Skoog', 'Geoffrey E. Hinton', 'Ellen E. Walters', 'Edward J. Roccella', 'Webb Miller', 'David G. Lowe', 'Katherine M. Flegal', 'Toby J. Gibson', 'John H. Holland', 'B. G. Barrell', 'Robert C. Onofrio']

WITH a, COUNT(q) AS num_top_collabs, COLLECT(DISTINCT b.name) as top_collabs
WHERE num_top_collabs > 3 

MATCH (a)-[:AUTHORED]->(q:Quanta)
WHERE q.venue in ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']
     AND exists(q.Disruption) AND a.last_author_yr <= q.year 
WITH a, top_collabs, COUNT(q) AS num_disrupt, SUM(q.Disruption) AS tot_disrupt, MIN(q.Disruption) AS min_disrupt, MAX(q.Disruption) AS max_disrupt, percentileCont(q.Disruption, 0.5) AS median_disrupt 
RETURN a.last_author_yr AS last_author_yr, 
    a.name AS name, 
    top_collabs,
    num_disrupt, 
    toFloat(tot_disrupt)/num_disrupt AS avg_disrupt, 
    min_disrupt, 
    max_disrupt, 
    median_disrupt"
, "/data/csv/Disrupt_Early_Profs_2.csv", {batchSize:100, iterateList:true, parallel:true})
"""
df_disruption = query_to_df(query, graph)
# df_disruption.to_csv('~/Workspace/UROP/scaling-science/notebooks/Collaboration/Data/Disrupt_Early_Profs.csv', index = False, encoding = "UTF-8")

Starting query... Done (0.01 minutes).


In [18]:
# Collaboration
avg_degree = 11.327052
query = """
call apoc.export.csv.query(
"MATCH (a:Top42Author)
WHERE a.last_author_yr > 2013 

MATCH (a)-[:AUTHORED]->(q:Quanta)<-[:AUTHORED]-(b:Author)
WHERE q.venue in ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']
    AND q.year < a.last_author_yr
    AND b.name in ['Timothy Fennell', 'Parnesh Raniga', 'Jeffrey C. Lee', 'James I. Cleeman', 'Todd M. Lowe', 'Amos Tversky', 'Eli Robins', 'Stephen T. Sherry', 'Desmond G. Higgins', 'John G. Kirkwood', 'Jane Peterson', 'Michael B. Eisen', 'G. E. Palade', 'Wayne J. Higgins', 'Sean R. Eddy', 'Torsten N. Wiesel', 'Jacques Ferlay', 'Nan M. Laird', 'Aravind Subramanian', 'Nathan Mantel', 'LaDeana W. Hillier', 'Richard Helm', 'Arian Smit', 'Anthony R. Kerlavage', 'Oliver H. Lowry', 'Carl-Magnus Rudenstam', 'P. Armitage', 'Nj Bell', 'Richard H. Creech', 'Russell Higuchi', 'Tobias Doerks', 'Theophil Staehelin', 'Edwin M. Southern', 'Sidney P. Colowick', 'Nicoletta Sacchi', 'J. Gillis', 'Stephen F. Altschul', 'Leon Festinger', 'I. M. Stratton', 'Julie D. Thompson', 'Thomas R. Dawber', 'Stanford Moore', 'E. Kellenberger', 'Kary B. Mullis', 'David A. Hungerford', 'Koichiro Tamura', 'Marshal F. Folstein', 'Tom Maniatis', 'Steven E. Scherer', 'C Redmond', 'Daniel Kahneman', 'Margaret D. Carroll', 'Elizabeth Lacy', 'David S. Johnson', 'Tavia Gordon', 'Ian Korf', 'David J. Dooling', 'R. A. Fisher', 'Marjorie B. Lees', 'Harry Towbin', 'William H. Stein', 'Paul Meier', 'Mark S. Guyer', 'Terrence S. Furey', 'Kara Dolinski', 'John P. Huelsenbeck', 'Jane G. Schaller', 'Susan E. Folstein', 'Randall K. Saiki', 'Ulrich K. Laemmli', 'William T. Friedewald', 'Michael L. Watson', 'F. Sanger', 'David P. Bartel', 'Rebecca DerSimonian', 'Felsenfeld Al', 'Michael C. Wendl', 'Saul A. Teukolsky', 'Parkhurst A. Shore', 'Josep F. Abril', 'David R. Bentley', 'N. E. Day', 'Kevin McKernan', 'Qui-Lim Choo', 'Tim R. Mosmann', 'Joseph L. Fleiss', 'Carole A. Cull', 'Thomas D. Schmittgen', 'Mark T. Ross', 'W. James Kent', 'Catherine A. Ball', 'Lucinda Fulton', 'Jeffrey Vieira', 'Patrick O. Brown', 'Kenneth J. Livak', 'Granger G. Sutton', 'David J. Lipman', 'Deanna M. Church', 'John E. Ware', 'Michael C. Zody', 'A. F. Huxley', 'Renaud Capdeville', 'A. L. Hodgkin', 'Sidney J. Cutler', 'Eugene W. Myers', 'Alan Coulson', 'Joseph Sambrook', 'Lars Hedin', 'Martin Gibson', 'Martin Ringwald', 'Marilyn Bergner', 'Masatoshi Nei', 'William Haenszel', 'D. R. Cox', 'R W Davis', 'Folke Skoog', 'Geoffrey E. Hinton', 'Ellen E. Walters', 'Edward J. Roccella', 'Webb Miller', 'David G. Lowe', 'Katherine M. Flegal', 'Toby J. Gibson', 'John H. Holland', 'B. G. Barrell', 'Robert C. Onofrio']

WITH a, COUNT(q) AS num_top_collabs, COLLECT(DISTINCT b.name) as top_collabs
WHERE num_top_collabs > 3

MATCH (a)-[:AUTHORED]->(q:Quanta)<-[:AUTHORED]-(b:Top42Author)
    WHERE q.venue in ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']
         AND a.last_author_yr <= q.year AND b.last_author_yr <= q.year  
    WITH a, top_collabs, b, 
        (toFloat(1) / (11.327052 + avg(q.num_profs_v2))) as u_lin,
        (toFloat(1) / sqrt(11.327052 + avg(q.num_profs_v2))) as u_sqrt,
        (toFloat(1) / (11.327052 + avg(q.num_profs_v2))^2) as u_square,
        (toFloat(1) / exp(11.327052 + avg(q.num_profs_v2))) as u_exp
        
    RETURN a.name as name, 
        top_collabs,
        a.last_author_yr as last_author_yr,
        count(u_lin) as unique_coauthor_profs, 
        11.327052*sum(u_lin) as unique_coauthor_profs_lin_damp,
        sqrt(11.327052)*sum(u_sqrt) as unique_coauthor_profs_sqrt_damp,
        11.327052^2*sum(u_square) as unique_coauthor_profs_square_damp,
        exp(11.327052)*sum(u_exp) as unique_coauthor_profs_exp_damp"
        , "/data/csv/Collab_Early_Profs_2.csv", {batchSize:100, iterateList:true, parallel:true})
    """
df_collab = query_to_df(query, graph)
# df_collab.to_csv('~/Workspace/UROP/scaling-science/notebooks/Collaboration/Data/Collab_Early_Profs.csv', index = False, encoding = "UTF-8")

Starting query... Done (0.03 minutes).


In [14]:
# Impact
query = """
call apoc.export.csv.query(
"MATCH (a:Top42Author)
WHERE a.last_author_yr > 2016 

MATCH (a)-[:AUTHORED]->(q:Quanta)<-[:AUTHORED]-(b:Author)
WHERE q.venue in ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']
    AND q.year < a.last_author_yr
    AND b.name in ['Timothy Fennell', 'Parnesh Raniga', 'Jeffrey C. Lee', 'James I. Cleeman', 'Todd M. Lowe', 'Amos Tversky', 'Eli Robins', 'Stephen T. Sherry', 'Desmond G. Higgins', 'John G. Kirkwood', 'Jane Peterson', 'Michael B. Eisen', 'G. E. Palade', 'Wayne J. Higgins', 'Sean R. Eddy', 'Torsten N. Wiesel', 'Jacques Ferlay', 'Nan M. Laird', 'Aravind Subramanian', 'Nathan Mantel', 'LaDeana W. Hillier', 'Richard Helm', 'Arian Smit', 'Anthony R. Kerlavage', 'Oliver H. Lowry', 'Carl-Magnus Rudenstam', 'P. Armitage', 'Nj Bell', 'Richard H. Creech', 'Russell Higuchi', 'Tobias Doerks', 'Theophil Staehelin', 'Edwin M. Southern', 'Sidney P. Colowick', 'Nicoletta Sacchi', 'J. Gillis', 'Stephen F. Altschul', 'Leon Festinger', 'I. M. Stratton', 'Julie D. Thompson', 'Thomas R. Dawber', 'Stanford Moore', 'E. Kellenberger', 'Kary B. Mullis', 'David A. Hungerford', 'Koichiro Tamura', 'Marshal F. Folstein', 'Tom Maniatis', 'Steven E. Scherer', 'C Redmond', 'Daniel Kahneman', 'Margaret D. Carroll', 'Elizabeth Lacy', 'David S. Johnson', 'Tavia Gordon', 'Ian Korf', 'David J. Dooling', 'R. A. Fisher', 'Marjorie B. Lees', 'Harry Towbin', 'William H. Stein', 'Paul Meier', 'Mark S. Guyer', 'Terrence S. Furey', 'Kara Dolinski', 'John P. Huelsenbeck', 'Jane G. Schaller', 'Susan E. Folstein', 'Randall K. Saiki', 'Ulrich K. Laemmli', 'William T. Friedewald', 'Michael L. Watson', 'F. Sanger', 'David P. Bartel', 'Rebecca DerSimonian', 'Felsenfeld Al', 'Michael C. Wendl', 'Saul A. Teukolsky', 'Parkhurst A. Shore', 'Josep F. Abril', 'David R. Bentley', 'N. E. Day', 'Kevin McKernan', 'Qui-Lim Choo', 'Tim R. Mosmann', 'Joseph L. Fleiss', 'Carole A. Cull', 'Thomas D. Schmittgen', 'Mark T. Ross', 'W. James Kent', 'Catherine A. Ball', 'Lucinda Fulton', 'Jeffrey Vieira', 'Patrick O. Brown', 'Kenneth J. Livak', 'Granger G. Sutton', 'David J. Lipman', 'Deanna M. Church', 'John E. Ware', 'Michael C. Zody', 'A. F. Huxley', 'Renaud Capdeville', 'A. L. Hodgkin', 'Sidney J. Cutler', 'Eugene W. Myers', 'Alan Coulson', 'Joseph Sambrook', 'Lars Hedin', 'Martin Gibson', 'Martin Ringwald', 'Marilyn Bergner', 'Masatoshi Nei', 'William Haenszel', 'D. R. Cox', 'R W Davis', 'Folke Skoog', 'Geoffrey E. Hinton', 'Ellen E. Walters', 'Edward J. Roccella', 'Webb Miller', 'David G. Lowe', 'Katherine M. Flegal', 'Toby J. Gibson', 'John H. Holland', 'B. G. Barrell', 'Robert C. Onofrio']
WITH a, COUNT(q) AS num_top_collabs, COLLECT(DISTINCT b.name) as top_collabs
WHERE num_top_collabs > 3

MATCH (a)-[:AUTHORED]->(q:Quanta)
WHERE a.last_author_yr > 2013 AND q.venue in ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']
    AND a.last_author_yr <= q.year 

WITH a, top_collabs, COUNT(q) AS num_pubs, 
    SUM(q.pageRank_2018) AS tot_rank, 
    MIN(q.pageRank_2018) AS min_rank, 
    MAX(q.pageRank_2018) AS max_rank, 
    percentileCont(q.pageRank_2018, 0.5) AS median_rank 

RETURN a.name AS name, 
    top_collabs,
    num_pubs,
    toFloat(tot_rank)/num_pubs AS avg_PR, 
    min_rank AS min_PR, 
    max_rank AS max_PR, 
    median_rank AS median_PR"
    , "/data/csv/PR_Early_Profs_2.csv", {batchSize:100, iterateList:true, parallel:true})

"""
df_pr = query_to_df(query, graph)

query = """
call apoc.export.csv.query(
"MATCH (a:Top42Author)
WHERE a.last_author_yr > 2013 

MATCH (a)-[:AUTHORED]->(q:Quanta)<-[:AUTHORED]-(b:Author)
WHERE q.venue in ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']
    AND q.year < a.last_author_yr
    AND b.name in ['Timothy Fennell', 'Parnesh Raniga', 'Jeffrey C. Lee', 'James I. Cleeman', 'Todd M. Lowe', 'Amos Tversky', 'Eli Robins', 'Stephen T. Sherry', 'Desmond G. Higgins', 'John G. Kirkwood', 'Jane Peterson', 'Michael B. Eisen', 'G. E. Palade', 'Wayne J. Higgins', 'Sean R. Eddy', 'Torsten N. Wiesel', 'Jacques Ferlay', 'Nan M. Laird', 'Aravind Subramanian', 'Nathan Mantel', 'LaDeana W. Hillier', 'Richard Helm', 'Arian Smit', 'Anthony R. Kerlavage', 'Oliver H. Lowry', 'Carl-Magnus Rudenstam', 'P. Armitage', 'Nj Bell', 'Richard H. Creech', 'Russell Higuchi', 'Tobias Doerks', 'Theophil Staehelin', 'Edwin M. Southern', 'Sidney P. Colowick', 'Nicoletta Sacchi', 'J. Gillis', 'Stephen F. Altschul', 'Leon Festinger', 'I. M. Stratton', 'Julie D. Thompson', 'Thomas R. Dawber', 'Stanford Moore', 'E. Kellenberger', 'Kary B. Mullis', 'David A. Hungerford', 'Koichiro Tamura', 'Marshal F. Folstein', 'Tom Maniatis', 'Steven E. Scherer', 'C Redmond', 'Daniel Kahneman', 'Margaret D. Carroll', 'Elizabeth Lacy', 'David S. Johnson', 'Tavia Gordon', 'Ian Korf', 'David J. Dooling', 'R. A. Fisher', 'Marjorie B. Lees', 'Harry Towbin', 'William H. Stein', 'Paul Meier', 'Mark S. Guyer', 'Terrence S. Furey', 'Kara Dolinski', 'John P. Huelsenbeck', 'Jane G. Schaller', 'Susan E. Folstein', 'Randall K. Saiki', 'Ulrich K. Laemmli', 'William T. Friedewald', 'Michael L. Watson', 'F. Sanger', 'David P. Bartel', 'Rebecca DerSimonian', 'Felsenfeld Al', 'Michael C. Wendl', 'Saul A. Teukolsky', 'Parkhurst A. Shore', 'Josep F. Abril', 'David R. Bentley', 'N. E. Day', 'Kevin McKernan', 'Qui-Lim Choo', 'Tim R. Mosmann', 'Joseph L. Fleiss', 'Carole A. Cull', 'Thomas D. Schmittgen', 'Mark T. Ross', 'W. James Kent', 'Catherine A. Ball', 'Lucinda Fulton', 'Jeffrey Vieira', 'Patrick O. Brown', 'Kenneth J. Livak', 'Granger G. Sutton', 'David J. Lipman', 'Deanna M. Church', 'John E. Ware', 'Michael C. Zody', 'A. F. Huxley', 'Renaud Capdeville', 'A. L. Hodgkin', 'Sidney J. Cutler', 'Eugene W. Myers', 'Alan Coulson', 'Joseph Sambrook', 'Lars Hedin', 'Martin Gibson', 'Martin Ringwald', 'Marilyn Bergner', 'Masatoshi Nei', 'William Haenszel', 'D. R. Cox', 'R W Davis', 'Folke Skoog', 'Geoffrey E. Hinton', 'Ellen E. Walters', 'Edward J. Roccella', 'Webb Miller', 'David G. Lowe', 'Katherine M. Flegal', 'Toby J. Gibson', 'John H. Holland', 'B. G. Barrell', 'Robert C. Onofrio']
WITH a, COUNT(q) AS num_top_collabs, COLLECT(DISTINCT b.name) as top_collabs
WHERE num_top_collabs > 3

MATCH (a)-[:AUTHORED]->(q:Quanta)
WHERE a.last_author_yr > 2013 AND q.venue in ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']
    AND a.last_author_yr <= q.year 

WITH a, top_collabs, COUNT(q) AS num_pubs, 
    SUM(q.articleRank2018) AS tot_rank, 
    MIN(q.articleRank2018) AS min_rank, 
    MAX(q.articleRank2018) AS max_rank, 
    percentileCont(q.articleRank2018, 0.5) AS median_rank 
    
RETURN a.name AS name, 
    top_collabs,
    toFloat(tot_rank)/num_pubs AS avg_AR, 
    min_rank AS min_AR, 
    max_rank AS max_AR, 
    median_rank AS median_AR"
    , "/data/csv/AR_Early_Profs_2.csv", {batchSize:100, iterateList:true, parallel:true})

"""
df_ar = query_to_df(query, graph)

Starting query... Done (0.01 minutes).
Starting query... Done (0.01 minutes).


In [5]:
avg_num_authors = 14.6906612
# Impact deweighted by num_authors
query = """
call apoc.export.csv.query(
"MATCH (a:Top42Author)
WHERE a.last_author_yr > 2013 

MATCH (a)-[:AUTHORED]->(q:Quanta)<-[:AUTHORED]-(b:Author)
WHERE q.venue in ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']
    AND q.year < a.last_author_yr
    AND b.name in ['Timothy Fennell', 'Parnesh Raniga', 'Jeffrey C. Lee', 'James I. Cleeman', 'Todd M. Lowe', 'Amos Tversky', 'Eli Robins', 'Stephen T. Sherry', 'Desmond G. Higgins', 'John G. Kirkwood', 'Jane Peterson', 'Michael B. Eisen', 'G. E. Palade', 'Wayne J. Higgins', 'Sean R. Eddy', 'Torsten N. Wiesel', 'Jacques Ferlay', 'Nan M. Laird', 'Aravind Subramanian', 'Nathan Mantel', 'LaDeana W. Hillier', 'Richard Helm', 'Arian Smit', 'Anthony R. Kerlavage', 'Oliver H. Lowry', 'Carl-Magnus Rudenstam', 'P. Armitage', 'Nj Bell', 'Richard H. Creech', 'Russell Higuchi', 'Tobias Doerks', 'Theophil Staehelin', 'Edwin M. Southern', 'Sidney P. Colowick', 'Nicoletta Sacchi', 'J. Gillis', 'Stephen F. Altschul', 'Leon Festinger', 'I. M. Stratton', 'Julie D. Thompson', 'Thomas R. Dawber', 'Stanford Moore', 'E. Kellenberger', 'Kary B. Mullis', 'David A. Hungerford', 'Koichiro Tamura', 'Marshal F. Folstein', 'Tom Maniatis', 'Steven E. Scherer', 'C Redmond', 'Daniel Kahneman', 'Margaret D. Carroll', 'Elizabeth Lacy', 'David S. Johnson', 'Tavia Gordon', 'Ian Korf', 'David J. Dooling', 'R. A. Fisher', 'Marjorie B. Lees', 'Harry Towbin', 'William H. Stein', 'Paul Meier', 'Mark S. Guyer', 'Terrence S. Furey', 'Kara Dolinski', 'John P. Huelsenbeck', 'Jane G. Schaller', 'Susan E. Folstein', 'Randall K. Saiki', 'Ulrich K. Laemmli', 'William T. Friedewald', 'Michael L. Watson', 'F. Sanger', 'David P. Bartel', 'Rebecca DerSimonian', 'Felsenfeld Al', 'Michael C. Wendl', 'Saul A. Teukolsky', 'Parkhurst A. Shore', 'Josep F. Abril', 'David R. Bentley', 'N. E. Day', 'Kevin McKernan', 'Qui-Lim Choo', 'Tim R. Mosmann', 'Joseph L. Fleiss', 'Carole A. Cull', 'Thomas D. Schmittgen', 'Mark T. Ross', 'W. James Kent', 'Catherine A. Ball', 'Lucinda Fulton', 'Jeffrey Vieira', 'Patrick O. Brown', 'Kenneth J. Livak', 'Granger G. Sutton', 'David J. Lipman', 'Deanna M. Church', 'John E. Ware', 'Michael C. Zody', 'A. F. Huxley', 'Renaud Capdeville', 'A. L. Hodgkin', 'Sidney J. Cutler', 'Eugene W. Myers', 'Alan Coulson', 'Joseph Sambrook', 'Lars Hedin', 'Martin Gibson', 'Martin Ringwald', 'Marilyn Bergner', 'Masatoshi Nei', 'William Haenszel', 'D. R. Cox', 'R W Davis', 'Folke Skoog', 'Geoffrey E. Hinton', 'Ellen E. Walters', 'Edward J. Roccella', 'Webb Miller', 'David G. Lowe', 'Katherine M. Flegal', 'Toby J. Gibson', 'John H. Holland', 'B. G. Barrell', 'Robert C. Onofrio']
WITH a, COUNT(q) AS num_top_collabs, COLLECT(DISTINCT b.name) as top_collabs
WHERE num_top_collabs > 3

MATCH (a)-[:AUTHORED]->(q:Quanta)
WHERE a.last_author_yr > 2013 AND q.venue in ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']
    AND a.last_author_yr <= q.year 
WITH a, top_collabs, COUNT(q) AS num_pubs, 
    SUM(14.6906612 * q.pageRank_2018 / (14.6906612 + q.num_authors)) as tot_rank,
    MIN(q.pageRank_2018) AS min_rank, 
    MAX(q.pageRank_2018) AS max_rank, 
    percentileCont(q.pageRank_2018, 0.5) AS median_rank 

RETURN a.name AS name, 
    top_collabs,
    num_pubs,
    toFloat(tot_rank)/num_pubs AS avg_PR, 
    min_rank AS min_PR, 
    max_rank AS max_PR, 
    median_rank AS median_PR"
    , "/data/csv/PR_Early_Profs_2.csv", {batchSize:100, iterateList:true, parallel:true})

"""
df_pr = query_to_df(query, graph)

query = """
call apoc.export.csv.query(
"MATCH (a:Top42Author)
WHERE a.last_author_yr > 2013 

MATCH (a)-[:AUTHORED]->(q:Quanta)<-[:AUTHORED]-(b:Author)
WHERE q.venue in ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']
    AND q.year < a.last_author_yr
    AND b.name in ['Timothy Fennell', 'Parnesh Raniga', 'Jeffrey C. Lee', 'James I. Cleeman', 'Todd M. Lowe', 'Amos Tversky', 'Eli Robins', 'Stephen T. Sherry', 'Desmond G. Higgins', 'John G. Kirkwood', 'Jane Peterson', 'Michael B. Eisen', 'G. E. Palade', 'Wayne J. Higgins', 'Sean R. Eddy', 'Torsten N. Wiesel', 'Jacques Ferlay', 'Nan M. Laird', 'Aravind Subramanian', 'Nathan Mantel', 'LaDeana W. Hillier', 'Richard Helm', 'Arian Smit', 'Anthony R. Kerlavage', 'Oliver H. Lowry', 'Carl-Magnus Rudenstam', 'P. Armitage', 'Nj Bell', 'Richard H. Creech', 'Russell Higuchi', 'Tobias Doerks', 'Theophil Staehelin', 'Edwin M. Southern', 'Sidney P. Colowick', 'Nicoletta Sacchi', 'J. Gillis', 'Stephen F. Altschul', 'Leon Festinger', 'I. M. Stratton', 'Julie D. Thompson', 'Thomas R. Dawber', 'Stanford Moore', 'E. Kellenberger', 'Kary B. Mullis', 'David A. Hungerford', 'Koichiro Tamura', 'Marshal F. Folstein', 'Tom Maniatis', 'Steven E. Scherer', 'C Redmond', 'Daniel Kahneman', 'Margaret D. Carroll', 'Elizabeth Lacy', 'David S. Johnson', 'Tavia Gordon', 'Ian Korf', 'David J. Dooling', 'R. A. Fisher', 'Marjorie B. Lees', 'Harry Towbin', 'William H. Stein', 'Paul Meier', 'Mark S. Guyer', 'Terrence S. Furey', 'Kara Dolinski', 'John P. Huelsenbeck', 'Jane G. Schaller', 'Susan E. Folstein', 'Randall K. Saiki', 'Ulrich K. Laemmli', 'William T. Friedewald', 'Michael L. Watson', 'F. Sanger', 'David P. Bartel', 'Rebecca DerSimonian', 'Felsenfeld Al', 'Michael C. Wendl', 'Saul A. Teukolsky', 'Parkhurst A. Shore', 'Josep F. Abril', 'David R. Bentley', 'N. E. Day', 'Kevin McKernan', 'Qui-Lim Choo', 'Tim R. Mosmann', 'Joseph L. Fleiss', 'Carole A. Cull', 'Thomas D. Schmittgen', 'Mark T. Ross', 'W. James Kent', 'Catherine A. Ball', 'Lucinda Fulton', 'Jeffrey Vieira', 'Patrick O. Brown', 'Kenneth J. Livak', 'Granger G. Sutton', 'David J. Lipman', 'Deanna M. Church', 'John E. Ware', 'Michael C. Zody', 'A. F. Huxley', 'Renaud Capdeville', 'A. L. Hodgkin', 'Sidney J. Cutler', 'Eugene W. Myers', 'Alan Coulson', 'Joseph Sambrook', 'Lars Hedin', 'Martin Gibson', 'Martin Ringwald', 'Marilyn Bergner', 'Masatoshi Nei', 'William Haenszel', 'D. R. Cox', 'R W Davis', 'Folke Skoog', 'Geoffrey E. Hinton', 'Ellen E. Walters', 'Edward J. Roccella', 'Webb Miller', 'David G. Lowe', 'Katherine M. Flegal', 'Toby J. Gibson', 'John H. Holland', 'B. G. Barrell', 'Robert C. Onofrio']
WITH a, COUNT(q) AS num_top_collabs, COLLECT(DISTINCT b.name) as top_collabs
WHERE num_top_collabs > 3

MATCH (a)-[:AUTHORED]->(q:Quanta)
WHERE a.last_author_yr > 2013 AND q.venue in ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']
    AND a.last_author_yr <= q.year 

WITH a, top_collabs, COUNT(q) AS num_pubs, 
    SUM(14.6906612 * q.articleRank2018 / (14.6906612 + q.num_authors)) as tot_rank,
    MIN(q.articleRank2018) AS min_rank, 
    MAX(q.articleRank2018) AS max_rank, 
    percentileCont(q.articleRank2018, 0.5) AS median_rank 
    
RETURN a.name AS name, 
    top_collabs,
    toFloat(tot_rank)/num_pubs AS avg_AR, 
    min_rank AS min_AR, 
    max_rank AS max_AR, 
    median_rank AS median_AR"
    , "/data/csv/AR_Early_Profs_2.csv", {batchSize:100, iterateList:true, parallel:true})

"""
df_ar = query_to_df(query, graph)

Starting query... Done (0.02 minutes).
Starting query... Done (0.01 minutes).


In [7]:
# Num affiliations for each author
query = """
call apoc.export.csv.query(
"MATCH (a:Top42Author)-[:AFFILIATED_WITH]->(o:Organization)
RETURN a.name as name, COUNT(o) as orgs",
"/data/csv/orgs.csv", {batchSize:100, iterateList:true, parallel:true})
"""
df = query_to_df(query, graph)

Starting query... Done (8.25 minutes).


In [ ]:
import pandas as pd
df_disrupt = pd.read_csv('~/scaling-science/data/magone/Disrupt_Early_Profs_2.csv', index_col = 'name', encoding = "UTF-8")
df_collab = pd.read_csv('~/scaling-science/data/magone/Collab_Early_Profs_2.csv', index_col = 'name', encoding = "UTF-8")
df_AR = pd.read_csv('~/scaling-science/data/magone/AR_Early_Profs_2.csv', index_col = 'name', encoding = "UTF-8")
df_PR = pd.read_csv('~/scaling-science/data/magone/PR_Early_Profs_2.csv', index_col = 'name', encoding = "UTF-8")
df_orgs = pd.read_csv('~/scaling-science/data/magone/orgs.csv', index_col = 'name', encoding = "UTF-8")


In [ ]:
def top_100_to_csv(trimmed_df, cols, name, num = 100, to_csv = False):
    d = dict()
    for col in [c for c in cols if c != 'top_collabs']:
        sorted = trimmed_df[[col, 'top_collabs']].sort_values(col, ascending = False).head(num)
        names = sorted.index.tolist()
        values = sorted[col].tolist()
        top_collabs = sorted['top_collabs'].tolist()
        d[col] = pd.Series(names, index=list(range(1, num+1)))
        d[col+'_val'] = pd.Series(values, list(range(1, num+1)))
        d[col+'_top_collabs'] = pd.Series(top_collabs, list(range(1, num+1)))
    df_rank = pd.DataFrame(d)
    if to_csv:
        df_rank.to_csv('~/scaling-science/data/magone/{}_Early_Profs_rank_2.csv'.format(name), index = False, encoding = "UTF-8")
    return df_rank

In [ ]:
num_pubs = df_PR[['num_pubs']]


In [ ]:
df_disrupt_trim = df_disrupt.join(df_orgs, how = 'inner')
print("Total: %d authors" % df_disrupt_trim.shape[0])
df_disrupt_trim = df_disrupt_trim[df_disrupt_trim['num_disrupt'] >= 3]
df_disrupt_trim = df_disrupt_trim[df_disrupt_trim['orgs'] <= 2000]
num = df_disrupt_trim.shape[0]
print("Remaining after trimming: %d authors" % num)
top_100_to_csv(df_disrupt_trim, df_disrupt.columns, 'Disrupt', num, to_csv = True)

In [ ]:
df_collab_trim = df_collab.join(df_orgs, how = 'inner')
df_collab_trim = df_collab_trim.join(num_pubs, how = 'inner')
print("Total: %d authors" % df_collab_trim.shape[0])
df_collab_trim = df_collab_trim[df_collab_trim['orgs'] <= 2000]
df_collab_trim = df_collab_trim[df_collab_trim['num_pubs'] >= 3]
num = df_collab_trim.shape[0]
print("Remaining after trimming: %d authors" % num)
top_100_to_csv(df_collab_trim, df_collab.columns, 'Collab', num, to_csv = True)

In [ ]:
df_AR_trim = df_AR.join(df_orgs, how = 'inner')
df_AR_trim = df_AR_trim.join(num_pubs, how = 'inner')
print("Total: %d authors" % df_AR_trim.shape[0])
df_AR_trim = df_AR_trim[df_AR_trim['orgs'] <= 2000]
df_AR_trim = df_AR_trim[df_AR_trim['num_pubs'] >= 3]
num = df_AR_trim.shape[0]
print("Remaining after trimming: %d authors" % num)
top_100_to_csv(df_AR_trim, df_AR.columns, 'AR', num, to_csv = True)

In [ ]:
df_PR_trim = df_PR.join(df_orgs, how = 'inner')
print("Total: %d authors" % df_PR_trim.shape[0])
df_PR_trim = df_PR_trim[df_PR_trim['orgs'] <= 2000]
df_PR_trim = df_PR_trim[df_PR_trim['num_pubs'] >= 3]
num = df_PR_trim.shape[0]
print("Remaining after trimming: %d authors" % num)
top_100_to_csv(df_PR_trim, df_PR.columns, 'PR', num, to_csv = True)